# Perform a nuclear segmentation on the DAPI staining using cellpose 2

In [ ]:
import imageio as io
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np
import tifffile
from tqdm.notebook import tqdm
import pathlib
from cellpose import models, core
import json

xenium_path = 'data'

DEVELOPMENT = os.getenv('DEVELOPMENT')

## Read in Xenium DAPI

In this part we import the DAPI OME TIFF, create a max projection of the different layers and downsample the image slightly.

In [ ]:
def read_dapi_image(path: str, downscale_factor: int = 2) -> np.ndarray:
    img_fpath = pathlib.Path(os.path.join(path, 'morphology_mip.ome.tif'))
    tif = tifffile.TiffFile(img_fpath)
    img = tif.asarray()
    if DEVELOPMENT:
        print(img.shape)
        img = img[10000:12000,10000:12000]
    return downscale_image(img, downscale_factor=downscale_factor)

def downscale_image(img: np.ndarray, downscale_factor: int = 2) -> np.ndarray:
    # Calculate the amount of padding needed for each axis
    pad_height = (downscale_factor - img.shape[0] % downscale_factor) % downscale_factor
    pad_width = (downscale_factor - img.shape[1] % downscale_factor) % downscale_factor

    # Pad the array with zeros
    img = np.pad(img, ((0, pad_height), (0, pad_width)), mode='constant')
    return img


In [ ]:
maxed_xenium = read_dapi_image(xenium_path, downscale_factor=1)

In [ ]:
maxed_xenium.shape

## Run cellpose

Here, we use the pretrained model to perform a nuclear segmentation with cellpose.

    This part takes A LOT of memory. Make sure you have at least 200 free Gb for a 35000 x 35000 pixel area.

I'm putting the model in the models folder with all the notebooks

In [ ]:
def run_cellpose(img: np.ndarray, model_path: str) -> (np.ndarray, np.ndarray, np.ndarray):
    use_GPU = core.use_gpu()
    model = models.CellposeModel(gpu=use_GPU, pretrained_model= model_path  )
    channels = [0,0]
    masks, flows, styles = model.eval([img], channels=channels, diameter=model.diam_labels,flow_threshold=0, cellprob_threshold=0)
    return (masks, flows, styles)


In [ ]:
masks, flows, styles = run_cellpose(
    maxed_xenium,
    model_path = r'models/DAPI'
)

Plot and save segmentation

In [ ]:
if DEVELOPMENT:
    plt.imshow(masks[0])

## Add the new segmentation to the transcripts.csv

In [ ]:
detected_transcripts = pd.read_csv(os.path.join(xenium_path, 'transcripts.csv.gz'))
detected_transcripts

Get the pixel to um conversion

In [ ]:
def get_pixel_size(path: str) -> float:
    file = open(os.path.join(path, "experiment.xenium"))
    experiment = json.load(file)
    pixel_size = experiment['pixel_size']
    return pixel_size

pixel_size = get_pixel_size(xenium_path)
pixel_size

In [ ]:
detected_transcripts['x_location_pixels'] = detected_transcripts.x_location.values*(1/pixel_size)
detected_transcripts['y_location_pixels'] = detected_transcripts.y_location.values*(1/pixel_size)

if DEVELOPMENT:
    detected_transcripts = detected_transcripts[
        (detected_transcripts['x_location_pixels'] > 10000) &
        (detected_transcripts['x_location_pixels'] < 12000) &
        (detected_transcripts['y_location_pixels'] > 10000) &
        (detected_transcripts['y_location_pixels'] < 12000)
    ].copy()
    detected_transcripts['x_location_pixels'] = detected_transcripts['x_location_pixels'] - 10000
    detected_transcripts['y_location_pixels'] = detected_transcripts['y_location_pixels'] - 10000

    detected_transcripts['x_location'] = detected_transcripts.x_location_pixels.values*(pixel_size)
    detected_transcripts['y_location'] = detected_transcripts.y_location_pixels.values*(pixel_size)

In [ ]:
detected_transcripts

In [ ]:
detected_cells = masks[0][detected_transcripts.y_location_pixels.values.astype(int), detected_transcripts.x_location_pixels.values.astype(int)]
detected_transcripts['cell_id'] = detected_cells
detected_transcripts['overlaps_nucleus'] = (detected_cells > 0).astype(int)
detected_transcripts

In [ ]:
detected_transcripts.to_csv("transcripts_cellpose.csv")